In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
import os
client = OpenAI(
    api_key = "ENTER YOUR OPENAI KEY HERE"
)

In [ ]:
prompt = """You will be provided with a list of tools. These tools have a description and a list of parameters and return types for each tool. Your task involves creating ten varied, complex, and detailed user scenarios that require at least two tool calls to complete involving at least two different tools. One of these tools will be explicitly provided and the others will be chosen by you. If the output of an ith tool is to be used as an argument value in some other tool then to reference the value of the ith tool in the chain, use $$PREV[i] as argument value. i = 0, 1, .. j-1; j = current tool’s index in the array. If the query could not be answered with the given set of tools, output an empty list instead.

For instance, given the tools: works_list, get_work_id, search_object_by_name, get_similar_work_items, summarize_objects.

Given that get_similar_work_items is explicitly provided, your scenario should articulate something akin to:

"Scenario: Summarize work items similar to don:core:dvrv-us-1:devo/0:issue/1
Required tools: [
        {
            "tool_name": "get_similar_work_items",
            "arguments": [
                    {
                        "argument_name": "work_id",
                        "argument_value": "don:core:dvrv-us-1:devo/0:issue/1"
                    }
            ]
        },
        {
            "tool_name": "summarize_objects",
            "arguments": [
                    {
                        "argument_name": "objects",
                        "argument_value": "$$PREV[0]"
                    }
            ]
        }
]"

This scenario exemplifies a scenario using two tool calls. The scenario is complex, detailed, and concise as desired. The scenario also includes two tools used in tandem, the required tools, get_similar_work_items to return a list of work items that are similar to don:core:dvrv-us-1:devo/0:issue/1 and summarize_objects to summarize a
list of objects. Usage of multiple tools in tandem is highly desirable and will receive a higher score. Ideally each scenario should contain one or more instances of multiple tools being used in tandem.
Note that this scenario does not use all the tools given. Re-using tools is allowed, but each scenario should involve at least two different tools. Note that tool usage is also included
in the scenario, but exact parameters are not necessary. You must use a different combination of tools for each scenario. You should provide a detailed reasoning for why you selected that set of tools and how you placed the arguments. All tools must be used in at least one scenario. You can only use the tools provided in the following Tool Set:
Tool Set:
{
    works_list: {
        "description": "Returns a list of work items matching the request",
        "arguments": [
            {
                "name": "applies_to_part",
                "description": "Filters for work belonging to any of the provided parts",
                "type": "array of strings",
                "example": ["FEAT-123", "ENH-123", "PROD-123", "CAPL-123"]
            },
            {
                "name": "created_by",
                "description": "Filters for work created by any of these users",
                "type": "array of strings",
                "example": ["DEVU-123"]
            },
            {
                "name": "issue.priority",
                "description": "Filters for issues with any of the provided priorities. Allowed values: p0, p1, p2, p3",
                "type": "array of strings",
                "example": ["p0"]
            },
            {
                "name": "issue.rev_orgs",
                "description": "Filters for issues with any of the provided Rev organizations",
                "type": "array of strings",
                "example": ["REV-123"]
            },
            {
                "name": "limit",
                "description": "The maximum number of works to return. The default is '50'",
                "type": "integer (int32)"
            },
            {
                "name": "owned_by",
                "description": "Filters for work owned by any of these users",
                "type": "array of strings",
                "example": ["DEVU-123"]
            },
            {
                "name": "stage.name",
                "description": "Filters for records in the provided stage(s) by name",
                "type": "array of strings",
                "example": ["triage"]
            },
            {
                "name": "ticket.needs_response",
                "description": "Filters for tickets that need a response",
                "type": "boolean"
            },
            {
                "name": "ticket.rev_org",
                "description": "Filters for tickets associated with any of the provided Rev organizations",
                "type": "array of strings",
                "example": ["REV-123"]
            },
            {
                "name": "ticket.severity",
                "description": "Filters for tickets with any of the provided severities. Allowed values: blocker, high, low, medium",
                "type": "array of strings",
                "example": ["high"]
            },
            {
                "name": "ticket.source_channel",
                "description": "Filters for tickets with any of the provided source channels",
                "type": "array of strings",
                "example": ["slack"]
            },
            {
                "name": "type",
                "description": "Filters for work of the provided types. Allowed values: issue, ticket, task",
                "type": "array of strings",
                "example": ["issue"]
            }
        ]
    },
    summarize_objects: {
        "description": "Summarizes a list of objects. The logic of how to summarize a particular object type is an internal implementation detail.",
        "arguments": [
            {
                "name": "objects",
                "description": "List of objects to summarize",
                "type": "array of objects"
            }
        ]
    },
    prioritize_objects: {
        "description": "Returns a list of objects sorted by priority. The logic of what constitutes priority for a given object is an internal implementation detail.",
        "arguments": [
            {
                "name": "objects",
                "description": "A list of objects to be prioritized",
                "type": "array of objects"
            }
        ]
    },
    "add_work_items_to_sprint": {
        "description": "Adds the given work items to the sprint",
        "arguments": [
            {
                "name": "work_ids",
                "description": "A list of work item IDs to be added to the sprint.",
                "type": "array of strings"
            },
            {
                "name": "sprint_id",
                "description": "The ID of the sprint to which the work items should be added",
                "type": "str"
            }
        ]
    },
    get_sprint_id: {
        "description": "Returns the ID of the current sprint",
        "arguments": []
    },
    get_similar_work_items: {
        "description": "Returns a list of work items that are similar to the given work item",
        "arguments": [
            {
                "name": "work_id",
                "description": "The ID of the work item for which you want to find similar items",
                "type": "string"
            }
        ]
    },
    search_object_by_name: {
        "description": "Given a search string, returns the id of a matching object in the system of record. If multiple matches are found, it returns the one where the confidence is highest.",
        "arguments": [
            {
                "name": "query",
                "description": "The search string, could be for example customer’s name, part name, user name.",
                "type": "string"
            }
        ]
    },
    create_actionable_tasks_from_text: {
        "description": "Given a text, extracts actionable insights, and creates tasks for them, which are kind of a work item.",
        "arguments": [
            {
                "name": "text",
                "description": "The text from which the actionable insights need to be created.",
                "type": "string"
            }
        ]
    },
    who_am_i: {
        "description": "Returns the ID of the current user",
        "arguments": []
    }
}

Deliver your response in the following JSON format. The given examples are only for three scenarios. You need to generate ten responses in this format. Remember that you are the user. Make sure that the exact value of the required arguments is present in the query. The query shouldn't be in the third person. Do not include the description of the tools or the chain of execution in the query in any way. The exact argument values that will be required for the tools should be present in the query. Keep the queries short. Strictly adhere to this format.
Give a detailed reasoning for why you selected those tools to solve that query.

- Scenario 1: <Scenario1>
Required Tools: [
                    {
                        "tool_name": { "type": "string" },
                        "arguments": [
                            {
                                "argument_name": { "type": "string" },
                                "argument_value": { "type": "string" }
                            }
                        ]
                    }
]
Reasoning:

- Scenario 2: <Scenario2>
Required Tools: [
                    {
                        "tool_name": { "type": "string" },
                        "arguments": [
                            {
                                "argument_name": { "type": "string" },
                                "argument_value": { "type": "string" }
                            }
                        ]
                    }
]
Reasoning:

- Scenario 3: <Scenario3>
Required Tools: [
                    {
                        "tool_name": { "type": "string" },
                        "arguments": [
                            {
                                "argument_name": { "type": "string" },
                                "argument_value": { "type": "string" }
                            }
                        ]
                    }
]
Reasoning:

"""

In [ ]:
response = client.chat.completions.create(
  model="gpt-4-1106-preview",
  messages=[
    {"role": "system", "content": "You are a tool-augmented llm"},
    {"role": "user", "content": prompt},
  ]
)

In [ ]:
print(response.choices[0].message.content)

- Scenario 1: I need to create tasks from a meeting notes text and then add those tasks to the current sprint.
Required Tools: [
                    {
                        "tool_name": "create_actionable_tasks_from_text",
                        "arguments": [
                            {
                                "argument_name": "text",
                                "argument_value": "Discuss integration with client's API, finalize the Q2 roadmap, and update the risk log."
                            }
                        ]
                    },
                    {
                        "tool_name": "add_work_items_to_sprint",
                        "arguments": [
                            {
                                "argument_name": "work_ids",
                                "argument_value": "$$PREV[0]"
                            },
                            {
                                "argument_name": "sprint_id",
                           